In [ ]:
from __future__ import print_function, division

import numpy as np

%matplotlib inline

#import matplotlib.pyplot as plt

from glowing_waffles.io import parse_aij_table

from glowing_waffles.visualization import multi_night

from astropy.table import Table, Column

## Set the name of your file in the cell below

The function below can read in either a raw AIJ file or csv files with the same information.

In [ ]:
sources = parse_aij_table('')

## Set EITHER the list of known variables or the list of comparison stars

Do **NOT** set both.

The one you do NOT want to use should be set to `None`

In [ ]:
# List of stars you think ARE variable 
known_variables = None # Change to list of stars in the field that you know vary, or None

# List of stars you think ARE NOT variable
keepers = [4, 9, 64, 103] # Change to list of comparisons you want to use, or None

# Check that settings are sensible

if known_variables is None and keepers is None:
    raise ValueError('You must set either known_variables or keepers to a list')

if known_variables is not None and keepers is not None:
    raise ValueError('Only ONE of known_variables and keeps can be set. '
                     'The other should be set to None.')

## Calculate comparison counts, AIJ-style

Which is to say that we calculate the comparison counts as the unweighted sum of the counts of all of the comparison stars, exceluding any known variables from the set.

In [ ]:
comp_counts = np.zeros_like(sources[0].counts)
for source in sources:
    if keepers:
        if source.id not in keepers: 
            # Variables shouldn't be in the comparison set
            continue
    elif known_variables:
        if source.id in known_variables:
            # Variables should not be in comparison set
            continue
    comp_counts += source.counts

In [ ]:
gain = 1.5

In [ ]:
mags = []
mag_err = []
for source in sources:
    mag = -2.5 * np.log10(source.counts / comp_counts)
    err = source.error 
    snr = gain * source.counts / err
    mag_e = 1/snr
    mags.append(mag)
    mag_err.append(mag_e)

### Find unique date for each night

We need to do this to group the data by night.

In [ ]:
night = np.floor(sources[0].bjd_tdb + 0.5) - 1

unique_nights = np.unique(night)

unique_nights = set(unique_nights) #- set([2457072.0])
unique_nights = sorted(unique_nights)
number_of_nights = len(unique_nights)
print(set(unique_nights))

In [ ]:
means = [np.median(mag[np.isfinite(mag)]) for mag in mags]

## Use second brightest star to scale brightness marker

In [ ]:
means.sort()
brightest_mag = np.max([means[1], -10])
print(brightest_mag)

# Plot the nights

In [ ]:
multi_night(sources, unique_nights, night,
            brightest_mag, mags, mag_err)